<a href="https://colab.research.google.com/github/noobcoder36/LSAP_usingColab/blob/main/CNN4_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Testing Trained Model with Test Data Set and Applying Greedy Collision


---



**Importing Libraries**

In [1]:
import numpy as np
# from scipy.optimize import linear_sum_assignment
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models



**Connecting Drive to Access Trained Model and Dataset**

In [2]:
# Accessing Drive
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "LSAP/CNN4/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # # create a test file to make sure it shows up in the right place
  # !touch 'new_file_in_working_directory.txt'
  # print('\nYour working directory was changed to ' + root_dir + project_folder + \
  #       "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

Mounted at /content/drive


**Loading Dataset**

In [3]:
# Reloding Data
with np.load('/content/drive/My Drive/LSAP/data_f.npz') as data:
  C = data['C']
  C_test = data['C_test']
  X_t1=data['X_t1']
  X_t2=data['X_t2']
  X_t3=data['X_t3']
  X_t4=data['X_t4']
  X_1=data['X_1']
  X_2=data['X_2']
  X_3=data['X_3']
  X_4=data['X_4']
  X_test = data['X_test']

n    = 4      # No. of Jobs and People 
m    = 45000   # No, of training samples 
test = 5000   # No, of test samples

**Reshaping Dataset to feed into Model**

In [4]:
BATCH_SIZE = 1024
SHUFFLE_BUFFER_SIZE = 256

C = np.repeat(C[..., np.newaxis], 1, -1)
C_test = np.repeat(C_test[..., np.newaxis], 1, -1)

test_dataset1 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t1))
test_dataset1 = test_dataset1.batch(BATCH_SIZE)

test_dataset2 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t2))
test_dataset2 = test_dataset2.batch(BATCH_SIZE)

test_dataset3 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t3))
test_dataset3 = test_dataset3.batch(BATCH_SIZE)

test_dataset4 = tf.data.Dataset.from_tensor_slices((C_test/100, X_t4))
test_dataset4 = test_dataset4.batch(BATCH_SIZE)


**Loading Trained Model**

In [5]:
# Saved Changes on Dec 1 
# Things to clarified; Training multpile models on the same graph and session and loading in the same graph and session
# Will this lead to wrong results
# To be done
# Interpret ouputs [[g1 g2 g3 g4]] 5000*4*4 and Get Xj and Apply Greedy Collision Rule{if Cij1 < Cij2: assign job i to person j1 } and Get the Inferred Final Ouput
# Now get accuracy of the results with corresponding Hungarain Algo's Output
# Saving Model will find its use when implemented on PC than on colab. As colab runs as if everything is on same script.

#1
n_model1 = tf.keras.models.load_model('saved_model/model1')

# Check its architecture
n_model1.summary()
g1= n_model1.predict(test_dataset1)

#2
n_model2 = tf.keras.models.load_model('saved_model/model2')

# Check its architecture
n_model2.summary()
g2= n_model2.predict(test_dataset2)

#3
n_model3 = tf.keras.models.load_model('saved_model/model3')

# Check its architecture
n_model3.summary()
g3= n_model3.predict(test_dataset3)


#4
n_model4 = tf.keras.models.load_model('saved_model/model4')
# Check its architecture
n_model4.summary()
g4 = n_model4.predict(test_dataset4)
g41= n_model4.evaluate(test_dataset4)
n_model1.evaluate(test_dataset1)
n_model2.evaluate(test_dataset2)
n_model3.evaluate(test_dataset3)

print('Predict:{}'.format(g4[0]))
print('Evaluate:{}'.format(g41[0]))
print(g41)

print(g1.shape)
print(g2.shape)
print(g3.shape)
print(g1[0])
print(g2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 4, 4, 2)           4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 4)           12        
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 17,684
Trainable params: 17,684
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 O

**Applying Greedy Collision Rule**

In [6]:
n=4
test = 5000
X_cap = np.zeros((test,n,n),dtype=int)
job = np.zeros((test,n),dtype=int)

p= [0, 1, 2, 3] # Persons {0..j}

count = np.zeros((test,1),dtype=int)

for s in range(test):
    job[s,p[0]] = np.argmax(g1[s])
    job[s,p[1]] = np.argmax(g2[s])
    job[s,p[2]] = np.argmax(g3[s])
    job[s,p[3]] = np.argmax(g4[s])

for s in range(test):
  for i in range(n-1):
    for j in range(i+1,n):
      if (job[s,p[i]]==job[s,p[j]]):
        count[s]+=1

k=0
sum1=0
sum2=0
sum3=0
sum4=0
sum5=0
sum6=0

for s in range(test):
  if(count[s]==0):
    k+=1
  if(count[s]==1):
    sum1+=1
  if(count[s]==2):
    sum2+=1
  if(count[s]==3):
    sum3+=1
  if(count[s]==4):
    sum4+=1
  if(count[s]==5):
    sum5+=1
  if(count[s]==6):
    sum6+=1

print('Collsion Count--> 0: {}| 1: {} | 2: {} | 3: {} | 4: {} | 5: {} | 6: {}'.format(k,sum1,sum2,sum3,sum4,sum5,sum6))

for s in range(test):
  for i in range(n):
    X_cap[s,job[s,p[i]],p[i]] = 1

print('Before Greedy:\n {}'.format(X_cap[0]))

Collsion Count--> 0: 3794| 1: 1198 | 2: 7 | 3: 1 | 4: 0 | 5: 0 | 6: 0
Before Greedy:
 [[0 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 1 0]]


In [7]:
r = np.zeros((test,1),dtype=int)
for s in range(test):
  if count[s]==1:

    for ind1 in range(n):
      r_s = X_cap[s,ind1,:] == np.zeros((n,),dtype=int)
      if (r_s.all()):
        r[s]=ind1
  else: 
    r[s] = -1

print('Before:\n {}'.format(X_cap[0:10]))
# print('Rows----------')
# print(r[0:10])

Before:
 [[[0 0 0 0]
  [1 0 0 0]
  [0 0 0 1]
  [0 1 1 0]]

 [[1 0 0 0]
  [0 1 0 0]
  [0 0 0 1]
  [0 0 1 0]]

 [[0 1 0 0]
  [0 0 0 1]
  [0 0 1 0]
  [1 0 0 0]]

 [[0 0 0 0]
  [1 0 0 0]
  [0 0 1 1]
  [0 1 0 0]]

 [[1 0 0 0]
  [0 1 0 0]
  [0 0 1 0]
  [0 0 0 1]]

 [[0 0 0 1]
  [0 1 0 0]
  [1 0 0 0]
  [0 0 1 0]]

 [[0 0 1 0]
  [0 0 0 1]
  [0 1 0 0]
  [1 0 0 0]]

 [[0 0 0 0]
  [0 0 0 1]
  [1 1 0 0]
  [0 0 1 0]]

 [[0 1 0 0]
  [1 0 0 0]
  [0 0 0 1]
  [0 0 1 0]]

 [[0 1 0 0]
  [0 0 1 0]
  [1 0 0 1]
  [0 0 0 0]]]


In [8]:
print(count[0:10])
print('Zero row {}'.format(r[0]))
print('Before Greedy:\n {}'.format(X_cap[0]))
for s in range(test):
  if count[s]==1:
    for i in range(n-1):
      for j in range(i+1,n):
        #Greedy Collison Rule  
        if (job[s,p[i]]==job[s,p[j]]):
            if (C_test[s,job[s,p[i]],p[i]] + C_test[s,r[s],p[j]] < C_test[s,job[s,p[j]],p[j]] + C_test[s,r[s],p[i]]):
                X_cap[s,job[s,p[i]],p[i]] = 1
                X_cap[s,job[s,p[j]],p[j]] = 0
                X_cap[s,r[s],p[j]]        = 1
                X_cap[s,r[s],p[i]]        = 0
            else:
                X_cap[s,job[s,p[j]],p[j]] = 1
                X_cap[s,job[s,p[i]],p[i]] = 0
                X_cap[s,r[s],p[i]]        = 1
                X_cap[s,r[s],p[j]]        = 0

print('After Greedy:\n {}'.format(X_cap[0]))

# Testing Accuracy 
# error = 0
# for s in range(test):
#   comparison = X_cap[s]  == X_test[s]
#   ind = comparison.all()
#   if not ind:
#     error+=1
err = np.zeros((n,1),dtype=int)
# ind = np.zeros((test,1),dtype=bool)
accuracy = np.ones((n,1),dtype=float)
for s in range(test):
  for i in range(n):
    comparison = X_cap[s,:,i]  == X_test[s,:,i]
    ind = comparison.all()
    if not ind:
      err[i]+=1

for i in range(n):
  accuracy[i] = (test-err[i])/test  
  
print('Error:\n{}'.format(err))
print('Accuracy:\n{}'.format(accuracy))
print('Accuracy Avg:{}'.format(sum(accuracy)/len(accuracy)))
print('Jobs for each person:{}'.format(job[0]))
print('Original Sol:\n{}'.format(X_test[0]))
print('Cost matrix 0th:\n{}'.format(C_test[0]))


[[1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]]
Zero row [0]
Before Greedy:
 [[0 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 1 0]]
After Greedy:
 [[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]
Error:
[[237]
 [246]
 [245]
 [238]]
Accuracy:
[[0.9526]
 [0.9508]
 [0.951 ]
 [0.9524]]
Accuracy Avg:[0.9517]
Jobs for each person:[1 3 3 2]
Original Sol:
[[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]
Cost matrix 0th:
[[[81]
  [ 2]
  [14]
  [99]]

 [[ 4]
  [48]
  [74]
  [58]]

 [[34]
  [77]
  [59]
  [34]]

 [[97]
  [59]
  [70]
  [50]]]


Time taken for CNN


In [9]:
import time
# starting time
start = time.time()
X_cap = np.zeros((test,n,n),dtype=int)
job = np.zeros((test,n),dtype=int)
p= [0, 1, 2, 3] # Persons {0..j}
count = np.zeros((test,1),dtype=int)
for s in range(test):
    job[s,p[0]] = np.argmax(g1[s])
    job[s,p[1]] = np.argmax(g2[s])
    job[s,p[2]] = np.argmax(g3[s])
    job[s,p[3]] = np.argmax(g4[s])
for s in range(test):
  for i in range(n-1):
    for j in range(i+1,n):
      if (job[s,p[i]]==job[s,p[j]]):
        count[s]+=1
for s in range(test):
  for i in range(n):
    X_cap[s,job[s,p[i]],p[i]] = 1
for s in range(test):
  if count[s]==1:
    for i in range(n-1):
      for j in range(i+1,n):
        #Greedy Collison Rule  
        if (job[s,p[i]]==job[s,p[j]]):
            if (C_test[s,job[s,p[i]],p[i]] + C_test[s,r[s],p[j]] < C_test[s,job[s,p[j]],p[j]] + C_test[s,r[s],p[i]]):
                X_cap[s,job[s,p[i]],p[i]] = 1
                X_cap[s,job[s,p[j]],p[j]] = 0
                X_cap[s,r[s],p[j]]        = 1
                X_cap[s,r[s],p[i]]        = 0
            else:
                X_cap[s,job[s,p[j]],p[j]] = 1
                X_cap[s,job[s,p[i]],p[i]] = 0
                X_cap[s,r[s],p[i]]        = 1
                X_cap[s,r[s],p[j]]        = 0
end = time.time()
# total time taken
print(f"Runtime of the program is {end - start}")

Runtime of the program is 0.11413025856018066


In [10]:
import time
# starting time
start = time.time()
g1= n_model1.predict(test_dataset1)
g2= n_model2.predict(test_dataset2)
g3= n_model3.predict(test_dataset3)
g4 = n_model4.predict(test_dataset4)
end = time.time()
# total time taken
print(f"Runtime of the program is {end - start}")

Runtime of the program is 0.09180402755737305


In [16]:
import time
# starting time
start = time.time()
g1= n_model1.predict(test_dataset1)
end = time.time()
# total time taken
print(f"Runtime of the program is {end - start}")

Runtime of the program is 0.03426003456115723
